In [1]:
import tensorflow as tf
print(tf.__version__)

1.12.0


In [2]:
h = tf.constant("Hello")
w = tf.constant("World!")
hw = h+w

with tf.Session() as sess:
    ans = sess.run(hw)

In [3]:
print(ans)

b'HelloWorld!'


In [4]:
print(hw)

Tensor("add:0", shape=(), dtype=string)


In [6]:
from tensorflow.examples.tutorials.mnist import input_data
# http://yann.lecun.com/exdb/mnist/ 에서 자료 내려받기

In [10]:
DATA_DIR = '/tmp/data'
NUM_STEPS = 1000
MINIBATCH_SIZE = 100

data = input_data.read_data_sets(DATA_DIR, one_hot=True)

x = tf.placeholder(tf.float32, [None, 784]) # placeholder는 연산 그래프가 실행될 때 제공되어야 하는 값
# x 는 이미지 자체는 연산 그래프가 실행될 때 제공되어야 하는 값이므로, placeholder
# [None, 784]의 의미는 한 이미지의 사이즈가 784(28*28)이며 얼마나 많은 이미지를 한 번에 사용할지는 이 시점에서 지정하지 않겠다는 의미(None)
W = tf.Variable(tf.zeros([784, 10])) # Variable이란 연산 과정에서 조작되는 값

y_true = tf.placeholder(tf.float32, [None, 10])
y_pred = tf.matmul(x, W)
# y_true = 정답 레이블
# y_pred = 예측 레이블

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y_pred, labels = y_true))

gd_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

correct_mask = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
accuracy = tf.reduce_mean(tf.cast(correct_mask, tf.float32))

with tf.Session() as sess:
    # 학습
    sess.run(tf.global_variables_initializer())
    
    for _ in range(NUM_STEPS):
        batch_xs, batch_ys = data.train.next_batch(MINIBATCH_SIZE)
        sess.run(gd_step, feed_dict={x: batch_xs, y_true: batch_ys})
    
    # 테스트
    ans = sess.run(accuracy, feed_dict = {x : data.test.images, y_true: data.test.labels})

print("Accuracy: {:.4}%".format(ans*100))

Extracting /tmp/data\train-images-idx3-ubyte.gz
Extracting /tmp/data\train-labels-idx1-ubyte.gz
Extracting /tmp/data\t10k-images-idx3-ubyte.gz
Extracting /tmp/data\t10k-labels-idx1-ubyte.gz
Accuracy: 91.59%


In [11]:
a = tf.constant(5)
b = tf.constant(2)
c = tf.constant(3)

In [12]:
d = tf.multiply(a,b)
e = tf.add(c,b)
f = tf.subtract(d,e)

In [14]:
sess = tf.Session()
outs = sess.run(f)
sess.close()
print("outs = {}".format(outs))

outs = 5


In [15]:
a = tf.constant(5)
b = tf.constant(2)

c = tf.multiply(a,b)
d = tf.add(a,b)

e = tf.subtract(c,d)
f = tf.add(c, d)

g = tf.divide(e, f)

In [16]:
sess = tf.Session()
outs = sess.run(g)
sess.close()
print("outs = {}".format(outs))

outs = 0.17647058823529413


In [18]:
print(tf.get_default_graph())

In [19]:
g = tf.Graph()
print(g)

In [20]:
g = tf.Graph()
a = tf.constant(5)

print(a.graph is g)
print(a.graph is tf.get_default_graph())

False
True


In [23]:
init_val = tf.random_normal((1,5),0,1)
var = tf.Variable(init_val, name='var')
print("pre run: \n{}".format(var))

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    post_var = sess.run(var)
    
print("\npost run: \n{}".format(post_var))

pre run: 
<tf.Variable 'var_2:0' shape=(1, 5) dtype=float32_ref>

post run: 
[[-0.16679144 -0.11375412 -1.4712716   1.5207409   1.0051429 ]]


In [25]:
import numpy as np

In [26]:
x_data = np.random.randn(5,10)
w_data = np.random.randn(10,1)

In [38]:
with tf.Graph().as_default():
    x = tf.placeholder(tf.float32, shape=(5,10))
    w = tf.placeholder(tf.float32, shape=(10,1))
    b = tf.fill([5,1],-1.)
    xw = tf.matmul(x,w)
    
    xwb = xw+b
    s = tf.reduce_max(xwb)
    with tf.Session() as sess:
        outs -sess.run(s, feed_dict={x:x_data, w:w_data})
    print("outs={}".format(outs))

outs=0.17647058823529413


# 예제 1

In [40]:
# 데이터를 생성하고 결과를 시뮬레이션
x_data = np.random.randn(2000,3)
w_real = [0.3,0.5,0.1]
b_real = -0.2

In [41]:
noise = np.random.randn(1,2000) * 0.1
y_data = np.matmul(w_real, x_data.T) + b_real + noise

In [45]:
NUM_STEPS = 10

g = tf.Graph()
wb_ = []
with g.as_default():
    x = tf.placeholder(tf.float32, shape=[None,3])
    y_true = tf.placeholder(tf.float32, shape = None)
    
    with tf.name_scope('inference') as scope:
        w = tf.Variable([[0,0,0]], dtype=tf.float32, name='weights')
        b = tf.Variable(0, dtype=tf.float32, name='bias')
        y_pred = tf.matmul(w, tf.transpose(x)) + b
    
    with tf.name_scope('loss') as scope:
        loss = tf.reduce_mean(tf.square(y_true-y_pred))
        
    with tf.name_scope('train') as scope:
        learning_rate = 0.5
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        train = optimizer.minimize(loss)
        
    # 시작하기 전에 변수를 초기화 한다
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)
        for step in range(NUM_STEPS):
            sess.run(train, {x: x_data, y_true: y_data})
            if (step % 5 == 0):
                print(step, sess.run([w,b]))
                wb_.append(sess.run([w,b]))
                
        print(10, sess.run([w,b]))

0 [array([[0.30490547, 0.52436113, 0.0895059 ]], dtype=float32), -0.20257393]
5 [array([[0.30069098, 0.49908292, 0.09771032]], dtype=float32), -0.20113444]
10 [array([[0.30069095, 0.49908292, 0.09771034]], dtype=float32), -0.20113446]


In [47]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1],
                         strides=[1,2,2,1], padding='SAME')

def conv_layer(input, shape):
    W = weight_variable(shape)
    b = bias_variable([shape[3]])
    return tf.nn.relu(conv2d(input, W) + b)

def full_layer(input, size):
    in_size = int(input.get_shape()[1])
    W = weight_variable([in_size, size])
    b = bias_variable([size])
    return tf.matmul(input, W) + b

In [50]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

x_image = tf.reshape(x, [-1, 28, 28, 1])
conv1 = conv_layer(x_image, shape=[5,5,1,32])
conv1_pool =  max_pool_2x2(conv1)

conv2 = conv_layer(conv1_pool, shape=[5,5,32,64])
conv2_pool = max_pool_2x2(conv2)

conv2_flat = tf.reshape(conv2_pool, [-1, 7*7*64])
full_1 = tf.nn.relu(full_layer(conv2_flat, 1024))

keep_prob = tf.placeholder(tf.float32)
full1_drop = tf.nn.dropout(full_1, keep_prob=keep_prob)

y_conv = full_layer(full1_drop, 10)